# 7. Attention机制
本节旨在介绍[Attention机制](https://zhuanlan.zhihu.com/p/46313756/)与两种典型的应用（Self-attention, Multi-head attention），及[基本的Attention实现方法](https://medium.com/intel-student-ambassadors/implementing-attention-models-in-pytorch-f947034b3e66)。

## 7.1 Attention
以NLP领域为例，常规的encoding是无法体现对一个句子序列中不同语素的关注程度的，然而一个句子中不同部分具有不同含义，并在意义上具有不同的重要性。

Attention机制是一种能让模型对重要信息重点关注并充分吸收的技术，能够作用于任何序列模型中。其通过赋予序列中不同语素以不同权重，结合实际场景的优化目标（如情感分析将着重关注Like/Dislike这种语素），来实现对不同语素进行不同侧重的目标。

### 7.1.1 Attention机制流程
**下面以seq2seq模型为例，阐述attention最基本的流程：**

对于一个包含有n个单词的句子序列source $S=[w_1, w_2, \cdots, w_n]$
1. 应用某种方法将 $S$ 的每个单词 $w_i$ 编码为一个单独的向量 $v_i$；
<p align=center>
<img src="./fig/7-3.png" width=700>
</p>

2. decoding阶段，使用学习到的Attention权重 $a_i$ 对1中得到的所有向量做线性加权 $\sum_i a_iv_i$。
<p align=center>
<img src="./fig/7-4.png" width=700>
</p>

3. 在decoder进行下一个单词的预测时，使用2中得到的线性组合。
<p align=center>
<img src="./fig/7-5.png" width=700>
</p>


由此可以抽象出Attention实现的三要素，Query，Key，Value，其中Q与K用于计算线性权重，V用于加权

<p align=center>
<img src="./fig/7-1.png" width=700>
</p>

对于Q, K, V的例子理解：

<p align=center>
<img src="./fig/7-2.png" width=700>
</p>

### 7.1.2 Attention的核心-注意力权重计算
Attention机制的核心在于如何通过Query和Key计算注意力权重，下面总结常用的几个方法：

1. 多层感知机(Multi-Layer Perception, MLP)
$$ a(q,k) = w_2^T tanh(W_1 [q;k])$$
首先将向量$q$与$k$进行拼接，经过全连接$W_1$线性映射后，$tanh$激活，通过一个全连接$w_2$线性映射至一个值。

MLP方法训练成本高，对大规模数据较为有效。

2. Bilinear
$$ a(q,k) = q^TWk$$
通过一个权重矩阵$W$建立$q$与$k$之间的相关关系，简单直接，计算速度快。

3. Dot Product
$$ a(q,k) = q^Tk$$
直接建立$q$与$k$之间的相关关系（内积，相似度），要求二者维度相同。

4. Scaled-dot Product
对3的改进，由于q和k的维度增加，会使得最后得到的内积a可能也会变得很大，这使得后续归一化softmax的梯度会非常小，不利于模型训练。参考[为什么dot-product需要被scaled](https://blog.csdn.net/qq_37430422/article/details/105042303)
$$ a(q,k) = \frac{q^Tk}{\sqrt{d_k}}$$
通过k的维度对a的尺度进行scaled，避免梯度消失问题。

## 7.2 Self-attention
Self-attention是attention机制的一种应用，其中，attention完成了输入source和输出target之间的加权映射。而self-attention字如其名，通过使得source=target，自己对自己本身进行注意力机制计算，来捕获序列数据自身的相互依赖特性。

即，在一般任务的Encoder-Decoder框架中，输入Source和输出Target内容是不一样的，比如对于英-中机器翻译来说，Source是英文句子，Target是对应的翻译出的中文句子，Attention机制发生在Target的元素Query和Source中的所有元素之间。

而Self-attention的注意力机制，是在Source=Target的特殊情况下，内部元素之间的attention机制，其具体计算过程是一样的，只是计算对象发生了变化而已。

<p align=center>
<img src="./fig/7-6.png" width=700>
</p>

如上图所示，我们将句子做self-attention，可以看到source中的语素'its'的attention集中在target中的语素'Law'与'application'上，这种self-attention使我们能够捕获这个句子内部不同元素间的依赖关系。

很明显，引入self-attention后，序列数据中长距离的相互依赖性将更容易被捕获。对于RNN来说，依次序列计算难以捕获远距离的依赖性，但self-attention通过直接将序列数据中任意两个样本的联系通过一个计算步骤直接联系起来，极大地缩短了远距离依赖性的距离，有利于有效地利用这些远距离相互依赖的特征。

## 7.3 Multi-head attention
另一种有效的应用为multi-head attention,其主要思想为从**多视角**看待（Q, K, V）的attention映射关系，是attention的拓展版本。

Multi-head attention通过设计h种不同的权重矩阵对 $(W_i^Q, W_i^K, W_i^V)_{i=1}^h$ 对 $ (Q, K, V)$进行attention计算，得到h个不同的 ${head_i}_{i=1}^h$ ，而后concat起来做一个全连接 $W^o$得到最后的attention输出，如图所示：

<p align=center>
<img src="./fig/7-7.png" width=300>
</p>

$$head_i = attention(QW^Q, KW^K, VW^V)$$
$$output = multihead(Q, K, V) = [head_1, \cdots, head_h]W^o$$

关于全连接 $(W_i^Q, W_i^K, W_i^V)$ 的输出维度 $(d^q, d^k, d^v)$，通常小于 $ (Q, K, V)$ 的输入维度 $d$，因为multi-head的计算成本过高，维度的增加将大大增加算法计算量，一般采用：

$$d^q=d^k=d^v=d/h$$

## 7.4 Attention机制的Torch实现
关于各种Attention的实现可以多加利用[github的轮子](https://github.com/xmu-xiaoma666/External-Attention-pytorch)，不得不说Github永远的神！

这里自己造个[简单的轮子](https://github.com/sooftware/attentions)，复现下Attention

### 7.4.1 Scaled Dot-Product Attention
Attention组件示意图：

<p align=center>
<img src="./fig/7-8.png" width=300>
</p>

In [6]:
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
import numpy as np
from typing import Optional

class ScaledDotProductAttention(nn.Module):
    """
    Args: dim
        - dim (int): dimension of attention (commonly, d_k).

    Inputs: query, key, value, mask
        - **query** (batch, q_len, d_model): tensor containing projection vector for decoder. d_model -> dimension of model (feature)
        - **key** (batch, k_len, d_model): tensor containing projection vector for encoder.
        - **value** (batch, v_len, d_model): tensor containing features of the encoded input sequence.
        - **mask** (batch, q_len, k_len): tensor containing indices to be masked.
    
    Outputs: context, attn
        - **context**: tensor containing the context vector from mechanism.
        - **attn**: tensor containing the attention from the encoder outputs.
    """
    def __init__(self, dim:int):
        super(ScaledDotProductAttention, self).__init__()
        self.sqrt_dim = np.sqrt(dim)
    
    def forward(self, query:Tensor, key:Tensor, value:Tensor, mask: Optional[Tensor] = None):
        # MatMul
        score = torch.bmm(query, key.transpose(1, 2)) # (batch, q_len, k_len)
        # Scale
        score = score / self.sqrt_dim
        # Mask (Opt)
        if mask is not None:
            score.masked_fill_(mask.view(score.size()), -float('Inf'))
        # Softmax
        attn = F.softmax(score, -1) # softmax along dimension "k_len"
        # MatMul
        context = torch.bmm(attn, value) # (batch, q_len, d_model)

        return context, attn

### 7.4.2 Multi-head Attention
Multi-head attention的组件示意图：

<p align=center>
<img src="./fig/7-9.png" width=300>
</p>

In [8]:
class MultiHeadAttention(nn.Module):
    """
    Project (q, k, v) h times with different, learned linear projections to d_head dimensions.

    Args: d_model, num_heads
        - d_model (int): The dimension of model (feature)
        - num_heads (int): The number of attention heads.
    
    Inputs: query, key, value, mask
        - **query** (batch, q_len, d_model)
        - **key** (batch, k_len, d_model)
        - **value** (batch, v_len, d_model)
        - **mask** (batch, q_len, k_len): tensor containing indices to be masked.

    Outputs: output, attn
        - **output** (batch, q_len, d_model): tensor containing the output features
        - **attn** (batch * num_heads, v_len): tensor containing the multi-head attention from the encoder outputs.
    """
    def __init__(self, d_model:int = 512, num_heads:int = 8):
        super(MultiHeadAttention,self).__init__()
        
        # Since d^q = d^k = d^v = d/h, d should be divided totally by h
        assert d_model % num_heads == 0, "Error: d_model % num_heads should be zero."
        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads

        # Instantiate a scaled dot-product attention object
        self.scaled_dot_attn = ScaledDotProductAttention(self.d_head)

        # Linear projection
        self.query_proj = nn.Linear(d_model, self.d_head * num_heads) # 'H' Linear Layers
        self.key_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.value_proj = nn.Linear(d_model, self.d_head * num_heads)

        # Linear
        self.Linear = nn.Linear(num_heads * self.d_head, d_model)
    
    def forward(self, query:Tensor, key:Tensor, value:Tensor, mask:Optional[Tensor] = None):
        batch_size = query.size(0)

        # Linear projection
        query = self.query_proj(query).view(batch_size, -1, self.num_heads, self.d_head) # (batch, q_len, num_heads, d_head)
        key = self.key_proj(key).view(batch_size, -1, self.num_heads, self.d_head)
        value = self.value_proj(value).view(batch_size, -1, self.num_heads, self.d_head)
        
        # Mask [Optional]
        if mask is not None:
            mask = mask.unsqueeze(1).repeat(1, self.num_heads, 1, 1) # BxHxQ_lenxK_len
        
        # Scaled dot-product attention
        query = query.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head) # (BxH, q_len, d_head)
        key = key.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head) # (BxH, q_len, d_head)
        value = value.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head) # (BxH, q_len, d_head)
        context, attn = self.scaled_dot_attn(query, key, value, mask)

        # Post-processing
        context = context.view(self.num_heads, batch_size, -1, self.d_head)
        context = context.permute(1, 2, 0, 3).contiguous().view(batch_size, -1, self.num_heads * self.d_head) # (B, q_len, Hxd_head)

        # Linear
        context = self.Linear(context)

        return context, attn
        

### 附录：关于squeeze/unsqueeze, Tensor.view的一些说明
1. squeeze为压缩操作（降维），unsqueeze为升维操作

In [27]:
x = torch.randn([1, 3, 4])
print('Before:', x.shape)
y = x.unsqueeze(0) # 在索引0处升维
print('Unsqueeze:', y.shape)

z = y.squeeze(0) # 在索引0处降维
print('Squeeze:', z.shape)

h = z.squeeze(0) # 在索引0处降维
print('Squeeze:', h.shape)

Before: torch.Size([1, 3, 4])
Unsqueeze: torch.Size([1, 1, 3, 4])
Squeeze: torch.Size([1, 3, 4])
Squeeze: torch.Size([3, 4])


然而，对于squeeze而言，其压缩操作只有当索引对应维度为1时才能生效，否则将不会做降维处理

In [29]:
x = torch.randn([1, 2, 3])
y = torch.randn([2, 2, 3])

x = x.squeeze(0)
print('x:', x.shape)

y = y.squeeze(0)
print('y:', y.shape)

x: torch.Size([2, 3])
y: torch.Size([2, 2, 3])


2. 对于Tensor.view()，当view的维度与Tensor的维度不一致的时候，将按照Tensor的元素顺序和view的维度进行重新切割

In [33]:
a = torch.Tensor([[[1,2,4], [2,3,4]]]) # Sequence: 1-2-4-2-3-4
a.shape
b = a.view([1,3,2]) # Now 1-2|4-2|3-4
print(b)
c = a.view([2,3]) # Now 1-2-4|2-3-4
print(c)

tensor([[[1., 2.],
         [4., 2.],
         [3., 4.]]])
tensor([[1., 2., 4.],
        [2., 3., 4.]])
